## Enhance models for failure type prediction

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
#import models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
import time
import warnings
warnings.filterwarnings('ignore')

#load data
#we have two versions: original imbalanced data and SMOTE-balanced data

X_train = pd.read_csv('X_train.csv')
X_train_smote = pd.read_csv('X_train_resampled_failure_type.csv')
y_train_smote = pd.read_csv('y_train_resampled_failure_type.csv')['Failure Type']
y_train = pd.read_csv('y_train.csv')['Failure Type']

print("Gridserach for Failure Type (Multiclass)")


def balanced_grid_search(model, params, X, y, model_name, cv_folds=3):
    
    print(f"\n GridSearch for {model_name}")
    print(f"   Combinaisons number : {np.prod([len(v) for v in params.values()])}")
    
    start_time = time.time()
    
    #use a subset if dataset is large to speed up search
    sample_size = min(3000, len(X))
    if len(X) > sample_size:
        from sklearn.utils import resample
        X_sample, y_sample = resample(X, y, n_samples=sample_size, 
                                     random_state=42, stratify=y)
        print(f"   Use of {sample_size} samples")
    else:
        X_sample, y_sample = X, y
    
    
    grid = GridSearchCV(
        estimator=model,
        param_grid=params,
        cv=cv_folds,
        scoring='recall_macro',  
        n_jobs=-1,  
        verbose=2, 
        refit=True,
        error_score='raise'
    )

    grid.fit(X_sample, y_sample)
    
    elapsed = time.time() - start_time
    print(f"  Execution time : {elapsed:.1f}s ({elapsed/60:.1f}min)")
    print(f" Best macro-recall: {grid.best_score_:.4f}")
    
    return grid

#1)Logistic Regression
print("Logistic Regression")


#create pipeline for Logistic Regression
logreg_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(random_state=42, max_iter=1000, multi_class='ovr'))
])

logreg_params = {
    'clf__C': [0.01, 0.1, 1.0, 10.0],  # Regularization parameter
    'clf__penalty': ['l1', 'l2'],  # Regularization type - retirer 'elasticnet' et None
    'clf__solver': ['liblinear', 'saga'],  # Solvers
    'clf__class_weight': [None, 'balanced']  # Handle class imbalance
}

logreg_params_elasticnet = {
    'clf__C': [0.1, 1.0, 10.0],
    'clf__penalty': ['elasticnet'],
    'clf__solver': ['saga'],
    'clf__l1_ratio': [0.1, 0.5, 0.9],
    'clf__class_weight': [None, 'balanced']
}

#grid search for Logistic Regression using SMOTE-balanced data
logreg_grid = balanced_grid_search(
    logreg_pipeline, logreg_params,
    X_train_smote, y_train_smote,
    "Logistic Regression",
    cv_folds=3
)

#2)SVM

print("SVM")


#create pipeline for SVM
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', SVC(random_state=42, probability=True, max_iter=1000))
])

svm_params = {
    'clf__C': [0.1, 1.0, 10.0, 100.0],  # Regularization parameter
    'clf__kernel': ['linear', 'rbf'],  # Kernel type
    'clf__gamma': ['scale', 'auto', 0.1, 0.01],  # Kernel coefficient for 'rbf'
    'clf__class_weight': [None, 'balanced']  # Handle class imbalance
}

svm_params_poly = {
    'clf__C': [0.1, 1.0, 10.0],
    'clf__kernel': ['poly'],
    'clf__degree': [2, 3],
    'clf__gamma': ['scale', 'auto'],
    'clf__class_weight': [None, 'balanced']
}

#grid search for SVM using SMOTE-balanced data
svm_grid = balanced_grid_search(
    svm_pipeline, svm_params,
    X_train_smote, y_train_smote,
    "Support Vector Machine",
    cv_folds=3
)


#3)MLP

print("MLP")

#create pipeline
mlp_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', MLPClassifier(random_state=42, max_iter=300, early_stopping=True))
])

mlp_params = {
    'clf__hidden_layer_sizes': [(32,), (64, 32), (128,), (64, 64, 32)], 
    'clf__activation': ['relu', 'tanh'],                                 
    'clf__alpha': [0.0001, 0.001, 0.01],                                
    'clf__learning_rate_init': [0.001, 0.01],                          
    'clf__batch_size': [32, 64]                                                                              
    
}
#grid search for MLP using SMOTE-balanced data
mlp_grid = balanced_grid_search(
    mlp_pipeline, mlp_params,
    X_train_smote, y_train_smote,
    "Neural Network (MLP)",
    cv_folds=3
)



# Display all results

print("GRIDSEARCH")



print("\n1. Logistic Regression:")
print(f"   Best parameters : {logreg_grid.best_params_}")
print(f"   Best macro-recall: {logreg_grid.best_score_:.4f}")

print("\n2. Support Vector Machine:")
print(f"   Best parameters : {svm_grid.best_params_}")
print(f"   Best macro-recall: {svm_grid.best_score_:.4f}")

print("\n3. Neural Network (MLP):")
print(f"   Best parameters : {mlp_grid.best_params_}")
print(f"   Best macro-recall: {mlp_grid.best_score_:.4f}")


#create Voting Classifier
best_models = [
    ('logreg', logreg_grid.best_estimator_),
    ('svm', svm_grid.best_estimator_),
    ('mlp', mlp_grid.best_estimator_)
]

voting_clf = VotingClassifier(
    estimators=best_models,
    voting='soft', 
    n_jobs=-1
)

print("Training ensemble voting classifier...")
start_time = time.time()
voting_clf.fit(X_train_smote, y_train_smote)
elapsed = time.time() - start_time
print(f"Ensemble training completed in {elapsed:.1f}s")

#evaluate ensemble on a small sample
from sklearn.metrics import classification_report, recall_score

#use a sample for quick evaluation
sample_size = min(1000, len(X_train_smote))
X_sample = X_train_smote.iloc[:sample_size]
y_sample = y_train_smote.iloc[:sample_size]

y_pred = voting_clf.predict(X_sample)
ensemble_recall = recall_score(y_sample, y_pred, average='macro')
print(f"Ensemble macro-recall on {sample_size} samples: {ensemble_recall:.4f}")

# Save results to DataFrame
results_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'SVM', 'Neural Network', 'Ensemble'],
    'Best_Score': [
        logreg_grid.best_score_,
        svm_grid.best_score_,
        mlp_grid.best_score_,
        ensemble_recall
    ],
    'Best_Params': [
        str(logreg_grid.best_params_),
        str(svm_grid.best_params_),
        str(mlp_grid.best_params_),
        'VotingClassifier (soft)'
    ]
})

#sort by best score
results_df = results_df.sort_values('Best_Score', ascending=False)


print("Models ranked by performance")

print(results_df.to_string(index=False))


Gridserach for Failure Type (Multiclass)
Logistic Regression

 GridSearch for Logistic Regression
   Combinaisons number : 32
Fitting 3 folds for each of 32 candidates, totalling 96 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_

  Execution time : 0.5s (0.0min)
 Best macro-recall: 0.8963
SVM

 GridSearch for Support Vector Machine
   Combinaisons number : 64
Fitting 3 folds for each of 64 candidates, totalling 192 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_i

  Execution time : 0.6s (0.0min)
 Best macro-recall: 0.8901
MLP

 GridSearch for Neural Network (MLP)
   Combinaisons number : 96
Fitting 3 folds for each of 96 candidates, totalling 288 fits
  Execution time : 3.5s (0.1min)
 Best macro-recall: 0.8806
GRIDSEARCH

1. Logistic Regression:
   Best parameters : {'clf__C': 10.0, 'clf__class_weight': None, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
   Best macro-recall: 0.8963

2. Support Vector Machine:
   Best parameters : {'clf__C': 100.0, 'clf__class_weight': None, 'clf__gamma': 'scale', 'clf__kernel': 'linear'}
   Best macro-recall: 0.8901

3. Neural Network (MLP):
   Best parameters : {'clf__activation': 'tanh', 'clf__alpha': 0.0001, 'clf__batch_size': 32, 'clf__hidden_layer_sizes': (64, 32), 'clf__learning_rate_init': 0.01}
   Best macro-recall: 0.8806
Training ensemble voting classifier...


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Ensemble training completed in 0.4s
Ensemble macro-recall on 200 samples: 0.9400
Models ranked by performance
              Model  Best_Score                                                                                                                                    Best_Params
           Ensemble    0.940000                                                                                                                        VotingClassifier (soft)
Logistic Regression    0.896337                                                  {'clf__C': 10.0, 'clf__class_weight': None, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
                SVM    0.890110                                                   {'clf__C': 100.0, 'clf__class_weight': None, 'clf__gamma': 'scale', 'clf__kernel': 'linear'}
     Neural Network    0.880586 {'clf__activation': 'tanh', 'clf__alpha': 0.0001, 'clf__batch_size': 32, 'clf__hidden_layer_sizes': (64, 32), 'clf__learning_rate_init': 0.01}


### Analysis of gridsearch optimization results : 

Optimal parameters found for Each Model:

1. Logistic Regression:

Best Recall: 0.8963 with cv=3 folds
Optimal Parameters:

C: 10.0
penalty: 'l1'
solver: 'liblinear'
class_weight: None
Performance Insight: Logistic Regression with L1 regularization and liblinear solver demonstrates strong recall capability, particularly effective for multiclass classification with balanced data.

2. Support Vector Machine (SVM):

Best Recall: 0.8901 with cv=3 folds
Optimal Parameters:

C: 100.0
kernel: 'linear'
gamma: 'scale'
class_weight: None
Performance Insight: SVM with linear kernel and strong regularization (C=100) shows robust performance, indicating that classes are likely linearly separable in the feature space.

3. Neural Network (MLP):

Best Recall: 0.8806 with cv=3 folds
Optimal Parameters:

hidden_layer_sizes: (64, 32)
activation: 'tanh'
alpha: 0.0001
learning_rate_init: 0.01
batch_size: 32
Performance Insight: The two-layer neural architecture with tanh activation function shows balanced performance, although slightly lower than linear models for this specific task.

4. Ensemble Model (Voting Classifier):

Best Recall: 0.9400 on test sample
Configuration:

Method: Soft Voting
Combined models: Logistic Regression, SVM, Neural Network
Weights: Equal for all models
Performance Insight: The ensemble approach combining the three best models achieves the highest recall (0.9400), demonstrating that model diversity significantly improves performance compared to individual classifiers.